# Tratamento da base de dados

In [ ]:
import pandas as pd
import numpy as np
base_casas = pd.read_csv('../csv-files/house_prices.csv')
base_casas

In [ ]:
base_casas.drop(columns=['id', 'date', 'sqft_living15', 'sqft_lot15'], axis=1, inplace=True)
base_casas

In [ ]:
# Dividindo os dados diretamente no pandas - Treinamento
base_treinamento = base_casas.iloc[0:15129,:]
base_treinamento.shape

In [ ]:
# Dividindo os dados diretamente no pandas - Teste
base_teste = base_casas.iloc[15129:, :]
base_teste.shape

Dados para treinamento são enviado no formato CSV Serializer
Dados para teste são enviados no formato numpy

In [ ]:
# Todos: da 1 a 17 . values (converte para p numpy)
X_teste = base_teste.iloc[:, 1:17].values
X_teste

In [ ]:
Y_teste = base_teste.iloc[:, 0].values
Y_teste

Transformando a base de treinamento para CSV

In [ ]:
# Necessario remover a coluna de Headers e os indices
# NECESSARIO QUE A COLUNA ALVO SEJA A PRIMEIRA
base_treinamento.to_csv('house_prices_train_xgboost.csv', header = False, index = False)
base_teste.to_csv('house_prices_test_xgboost.csv', header = False, index = False)

# Configuração SageMaker

In [ ]:
import boto3
import sagemaker
from sagemaker import Session

session = sagemaker.Session() # Iniciando session

bucket = 'sagemaker-curso-bucket' # Bucket S3
subpasta_modelo = 'modelos/house-prices/xgboost'
subpasta_dataset = 'datasets/house-prices'

key_train = 'house-train-data-xgboost'
key_test = 'house-test-data-xgboost'

role = "AmazonSageMaker-ExecutionRole-20240702T173175"

s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subpasta_dataset, key_train)
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, subpasta_dataset, key_test)

output_location = 's3://{}/{}/output'.format(bucket, subpasta_modelo)

print(s3_train_data)
print(s3_test_data)
print(output_location)

Transformando no tipo de arquivo aceito no sagemaker

In [ ]:
import os
with open('./house_prices_train_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'train', key_train)).upload_fileobj(f)

In [ ]:
with open('./house_prices_test_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'test', key_test)).upload_fileobj(f)

# Treinamento XGBoost

In [ ]:
from sagemaker import image_uris

container = sagemaker.image_uris.retrieve(framework='xgboost', region=boto3.Session().region_name, version='latest')

xgboost = sagemaker.estimator.Estimator(image_uri=container,
                                       role = role,
                                       instance_count= 1,
                                       instance_type='ml.m5.2xlarge',
                                       output_path= output_location,
                                       sagemaker_session= session
                                    )

In [ ]:
# Variaveis
xgboost.set_hyperparameters(num_round = 100)


No Linear Leaner o treinamento e a validacao sao feitos com a mesma base de dados. No XGBoost não.

In [ ]:
train_input = sagemaker.inputs.TrainingInput(s3_data=s3_train_data,
                                             content_type='csv',
                                             s3_data_type='S3Prefix')

validation_input = sagemaker.inputs.TrainingInput(s3_data=s3_test_data,
                                             content_type='csv',
                                             s3_data_type='S3Prefix')

data_channels = {
    "train": train_input,
    "validation": validation_input
}

In [ ]:
# Treinamento
xgboost.fit(data_channels)

# Deploy, previsões e avaliação

# Tuning

Como descobrir os parametros ideiais para o treinamento:

In [ ]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
""" {
          Faixa de teste
          "MaxValue": "1",
          "MinValue": "0",
          Nome do parametro 
          "Name": "eta"
}, """

tuning_job_config = {
    "ParameterRanges": {
      "CategoricalParameterRanges": [],
      "ContinuousParameterRanges": [
        {
          "MaxValue": "1",
          "MinValue": "0",
          "Name": "eta"
        },
        {
          "MaxValue": "2",
          "MinValue": "0",
          "Name": "alpha" # Regularização 
        },
        {
          "MaxValue": "10",
          "MinValue": "1",
          "Name": "min_child_weight" # Numero minimo de instancias para ser definido uma folha na arvore
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "10",
          "MinValue": "1",
          "Name": "max_depth" # Profundidade maxima da arvore
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "300",
          "MinValue": "50",
          "Name": "num_round" # Numero de rodadas de treinamento
        }
      ]
    },
    "ResourceLimits": { # Quanto poder computacional será usado
      "MaxNumberOfTrainingJobs": 9,
      "MaxParallelTrainingJobs": 3 # numero de algortimos rodando em paralelo
    },
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {
      "MetricName": "validation:rmse", # minimizar o valor do erro usando rmse
      "Type": "Minimize"
    }
  }

In [ ]:
training_job_definition = {
    "AlgorithmSpecification": {
      "TrainingImage": container, # Variavel criada
      "TrainingInputMode": "File"
    },
    "InputDataConfig": [
      {
        "ChannelName": "train",
        "CompressionType": "None",
        "ContentType": "csv",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated",
            "S3DataType": "S3Prefix",
            "S3Uri": s3_train_data # Caminho para os arquvios de trainemnto
          }
        }
      },
      {
        "ChannelName": "validation",
        "CompressionType": "None",
        "ContentType": "csv",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated",
            "S3DataType": "S3Prefix",
            "S3Uri": s3_test_data # caminho para os arquivos de teste
          }
        }
      }
    ],
    "OutputDataConfig": {
      "S3OutputPath": "s3://{}/{}/output".format(bucket,subpasta_modelo) # onde será salvo o modelo
    },
    "ResourceConfig": {
      "InstanceCount": 2,
      "InstanceType": "ml.c4.2xlarge",
      "VolumeSizeInGB": 10
    },
    "RoleArn": "arn:aws:iam::339712791663:role/service-role/AmazonSageMaker-ExecutionRole-20240702T173175",
    "StaticHyperParameters": {
      "eval_metric": "rmse",
      "objective": "reg:linear",
      "rate_drop": "0.3",
      "tweedie_variance_power": "1.4"
    },
    "StoppingCondition": {
      "MaxRuntimeInSeconds": 43200
    }
}

In [ ]:
smclient = boto3.client('sagemaker')
smclient.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = "xgboosttuninghouses",
                                          HyperParameterTuningJobConfig = tuning_job_config,
                                          TrainingJobDefinition = training_job_definition)

Contruindo novo modelo.

In [ ]:
container = image_uris.retrieve(framework='xgboost',region=boto3.Session().region_name,version='latest')
xgboost_tuning = sagemaker.estimator.Estimator(image_uri = container,
                                        role = role, 
                                        instance_count = 1, 
                                        instance_type = 'ml.m5.2xlarge',
                                        output_path = output_location,
                                        sagemaker_session = session)
# Valores coletados do console.
xgboost_tuning.set_hyperparameters(num_round = 215, eta = 0.07545286994225804,
                                   min_child_weight = 2.4061755279241996,
                                   alpha = 1.5934054040797325, tweedie_variance_power = 1.4,
                                   rate_drop = 0.3)

xgboost_tuning.fit(data_channels)

In [ ]:
xgboost_regressor_tuning.serializer = CSVSerializer()
previsoes = np.array(xgboost_regressor_tuning.predict(X_teste).decode('utf-8').split(',')).astype(np.float32)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_teste, previsoes)
mse = mean_squared_error(y_teste, previsoes)
print('MAE = ', mae, '\nMSE = ', mse)